In [65]:
import numpy as np
import random

generate unique delays

In [66]:
MIN_RANGE = 300.0
MAX_RANGE = 1200.0
COUNT = 50

prop_delays = set()
while len(prop_delays) < COUNT:
    prop_delays.add(random.uniform(MIN_RANGE, MAX_RANGE))

print(prop_delays)

{769.9189361548327, 515.0590995461168, 390.0581995261307, 648.3826488999765, 906.0587983096333, 511.96570685537625, 1034.0431353694569, 394.41309709573864, 526.7143551122302, 914.6996636711284, 663.1063254289479, 1176.2168696192243, 924.6397849714531, 1186.102387728827, 803.5158129207261, 804.1900550713592, 803.4799963443957, 422.3550788686649, 424.85204974670546, 937.3682719130429, 301.0307087313908, 691.0547582216725, 694.2662723422659, 310.0242096667472, 823.9770974238567, 311.65420410211203, 570.0932815014139, 824.7712678013365, 835.920471463488, 325.14388860211915, 714.4656984877497, 334.36358945041917, 1106.4862566774545, 469.84365983961663, 476.3816048682853, 349.31378182665594, 889.4569979733853, 866.9213753437682, 1124.1370401663914, 356.0131332924687, 1129.6810584429568, 747.6441702242962, 621.788111173679, 1135.7365083743773, 496.33986872409776, 370.4891747742262, 374.8413687507833, 633.758967408106, 1018.7737265704263, 1151.4500482354024}


select N random pairs of indexes

In [67]:
N = 40
pairs = set()

while len(pairs) < N:
    first = random.randint(0, COUNT - 1)
    second = random.randint(0, COUNT - 1)
    pairs.add((first, second))

print(pairs)

{(46, 30), (43, 46), (7, 26), (45, 16), (22, 20), (44, 48), (18, 35), (13, 39), (0, 2), (20, 29), (3, 22), (43, 3), (4, 30), (17, 27), (3, 40), (1, 12), (49, 16), (49, 13), (42, 41), (16, 8), (25, 40), (46, 10), (8, 38), (20, 3), (25, 30), (37, 1), (1, 38), (13, 40), (36, 11), (32, 47), (43, 1), (18, 36), (48, 43), (27, 14), (39, 40), (12, 20), (8, 25), (1, 19), (15, 18), (15, 15)}


generate bit vectors

In [68]:
bit_vector = list()
delays = list(prop_delays)


for i in pairs:
    x = delays[i[0]]
    y = delays[i[1]]

    res = 1 if x > y else 0
    bit_vector.append(res)

print(bit_vector)

[0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0]
